# Classification de photos de légumes

## Problèmatique

Le projet consiste à obtenir un modèle capable de classer une photo de légume.<br />

## Les données

Les données utilisées pour ce projet proviennent d'un dataset trouvé sur Kaggle.<br />
Source : https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset/data<br />
Le dataset a été téléchargé à la main au préalable, puis déposé dans le dossier resources.

## Modules

In [1]:
import nnz

w = nnz.Workspace()

In [2]:
X_train, y_train, X_val, y_val, X_test, y_test = w.split_folders("./resources/classes", "./resources", verbose=0, size_train=0.6, size_val=0.15, size_test=0.25, resize=(128,128))

In [3]:
w.get_classes()

,name,train_count,val_count,test_count
0,Bitter_Gourd,840,210,350
1,Cauliflower,840,210,350
2,Papaya,840,210,350
3,Tomato,840,210,350
4,Brinjal,840,210,350
5,Carrot,840,210,350
6,Capsicum,840,210,350
7,Potato,840,210,350
8,Bean,840,210,350
9,Bottle_Gourd,840,210,350


In [4]:
X_train.shape

(12600, 128, 128, 3)

In [5]:
y_train.shape

(12600,)

In [6]:
y_train[:10]

array(['Papaya', 'Bitter_Gourd', 'Potato', 'Bitter_Gourd', 'Potato',
       'Papaya', 'Carrot', 'Cabbage', 'Cucumber', 'Cauliflower'],
      dtype='<U12')

In [12]:
import cv2
import numpy as np
# cv2.imshow('image', X_train[6])
# cv2.waitKey(0)
# cv2.destroyAllWindows()
np.unique(y_train, return_counts=True)

(array(['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli',
        'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber',
        'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato'], dtype='<U12'),
 array([840, 840, 840, 840, 840, 840, 840, 840, 840, 840, 840, 840, 840,
        840, 840]))